In [ ]:

# from analysis.probability import probability_function
from evaluate import evaluate_single

probability_generator = lambda x: None # probability_function

def eval_simple(note):
    return evaluate_single(probability_generator(note), root_note=note)

In [ ]:
import numpy as np
from analysis.musical import consonance_ordered_notes, empirical_probabilities
from analysis.spike_tensor import generate_spike_tensor
from cache import get_spikes
from evaluate import predicted_consonance_scores, predicted_probabilities

root_note = "C4"
notes = consonance_ordered_notes(root_note)
notes_spikes = [get_spikes(note) for note in notes]
consonance_ordered_tensors = [generate_spike_tensor(spikes) for spikes in notes_spikes]

root_tensor = generate_spike_tensor(get_spikes(root_note))
probability_tensor = probability_generator(root_note)

def try_eval(tensors, metric, debug=False):
    scores = predicted_consonance_scores(probability_tensor, tensors, root_tensor)

    P_predicted = predicted_probabilities(scores)

    if debug:
        def print_arr(arr):
            print(np.round(np.array(arr), 3))
        print("Scores")
        print_arr(scores)
        print("P_predicted")
        print_arr(P_predicted)
        print("P_empirical")
        print_arr(empirical_probabilities())

    print(metric(P_predicted))

In [ ]:
from evaluate import js_divergence

try_eval(consonance_ordered_tensors, js_divergence, debug=True)

In [ ]:
import numpy as np

def collect_data(metric, N=10):
    data = []
    for i in range(N):
        A = consonance_ordered_tensors[0][0][0]
        np.random.shuffle(consonance_ordered_tensors)
        B = consonance_ordered_tensors[0][0][0]
        if not np.array_equal(A, B):
            value = try_eval(consonance_ordered_tensors, metric)
            data.append(value)

In [ ]:
from evaluate import js_divergence

collect_data(js_divergence)

In [ ]:
import os

from cache import SPIKES_DIR


root_note = "C4"
notes = consonance_ordered_notes(root_note)
notes = [notes[0], notes[-1]]
notes_spikes = [get_spikes(note) for note in notes]
tensors = [generate_spike_tensor(spikes) for spikes in notes_spikes]

white_noise_spikes = np.load(os.path.join(SPIKES_DIR, "white_noise.npy"))
tensors.append(generate_spike_tensor(white_noise_spikes))

root_tensor = generate_spike_tensor(get_spikes(root_note))

scores = predicted_consonance_scores(probability_tensor, tensors, root_tensor)
print("Scores")
print(np.round(np.array(scores)))

P_predicted = predicted_probabilities(scores)
print("P_predicted")
print(np.round(np.array(P_predicted), 2))

print("Most Cons, Least Cons, White Noise")


In [ ]:
keys = ["C4", "C#4", "D4", "D#4", "E4", "F4", "F#4", "G4", "G#4", "A4", "A#4", "B4"]

for key in keys:
    print(f"{key}: {eval_simple(key)}")

In [ ]:
consonance_ordered_notes(root_note)

In [ ]:
# collect_data(kl_divergence_reverse)
# collect_data(kl_divergence)